In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline
#%matplotlib notebook
%reload_ext autoreload
%autoreload 2

plt.rcParams['figure.figsize'] = 10, 5
np.set_printoptions(precision=5, suppress=True)

In [ ]:
from trajectory import Trajectory

np.random.seed(1)
t1 = Trajectory(dim=2, n_complexity=2, model='polynomial', 
                coeffs=np.array([[0, 0], [0, 1]]))
t2 = Trajectory(dim=2, n_complexity=2, model='polynomial', 
                coeffs=np.array([[-3, 3], [1, 0]]))
times1 = np.linspace(0, 1, 10)
times2 = np.linspace(1, 2, 20)
anchors = 4 * np.random.rand(2, 5)
fig, ax = plt.subplots()
ax.scatter(*anchors, color='red')
t1.plot(times=times1, ax=ax, color='black')
t2.plot(times=times2, ax=ax, color='black')
ax.axis('equal')

from measurements import get_measurements

b1, D1 = get_measurements(t1, anchors, seed=1, times=times1)
b2, D2 = get_measurements(t2, anchors, seed=1, times=times2)
F = np.hstack((b1, b2))
D = np.vstack((D1, D2))
times = np.r_[times1, times2]
print(D.shape, F.shape)

#plt.matshow(D)

# Averaging Algorithm

Estimate the trajectory recursively over a fixed  time window.

In [ ]:
from solvers import alternativePseudoInverse

T = 1.0

line_counter = 0

Dk = np.empty((0, D.shape[1]))
Fk = np.empty((F.shape[0], 0))
C_list = []
t_list = np.linspace(0, 2, 20)

fig, ax = plt.subplots()
for t_s in t_list:
    tk = times[(times>=t_s) & (times< t_s+T)]
    for t_n in tk:
        #d_mn, a_m, t = df.iloc[line_counter, 
        #                       ['distance', 'anchor_id', 'timestamp']]
        # d_mn_row = np.zeros(anchors.shape[1]) # pad with zeros.
        # d_mn_row[anchor_ids==a_m] = d_mn
        # line_counter += 1
        # f_n = t1.get_basis(times=[t]) 
        
        idx = np.where(times == t_n)[0]
        
        d_mn_row = D[idx, :]
        f_n = F[:, idx]
        
        Dk = np.r_[Dk, d_mn_row]
        Fk = np.c_[Fk, f_n]
        
    try:
        Ck = alternativePseudoInverse(Dk, anchors, Fk) 
        C_list.append(Ck)
    
        trajk = t1.copy()
        trajk.set_coeffs(coeffs=Ck)
        trajk.plot(times=tk, ax=ax)
        
    except:
        print('skipping {:.2f} because only'.format(t_s), tk)
    
    Dk = np.empty((0, D.shape[1]))
    Fk = np.empty((F.shape[0], 0))

# Algo 1


Refine the current trajectory as long as the new measurements "fit the model" well enough. 

Create a completely new trajectory model with measurements if they stop fitting. 

In [ ]:
Ck = None
tk = []
eps = 1e-3

Dk = np.empty((0, D.shape[1]))
Fk = np.empty((F.shape[0], 0))

fig, ax = plt.subplots()

C_list = []
t_list = []

def g():
    r_n = Ck.dot(f_n).reshape((ams.shape[0], 1))
    dist_estimates = np.linalg.norm(ams - r_n, axis=0)
    return np.sum(dist_estimates**2 - d_mn_row)

for d_mn_row, t_n, f_n in zip(D, times, F.T):
    
    d_mn_row = d_mn_row.reshape((1, -1))
    ams = anchors[:, d_mn_row[0]>0]
    ds = d_mn_row[0, d_mn_row[0] > 0]
    if Ck is None or g() <= eps:
        Dk = np.r_[Dk, d_mn_row]
        Fk = np.c_[Fk, f_n]
        
        try:
            Ck = alternativePseudoInverse(Dk, anchors, Fk) 
        except AssertionError:
            print('not ready yet.')
        #except Exception as e:
        #    raise e
        tk.append(t_n)
            
    elif (Ck is not None) and g() > eps:
        print('changing to new trajectory, because', g())
        
        Fk = f_n
        Dk = d_mn_row
        
        C_list.append(Ck)
        t_list.append(tk)
        
        
        print('plotting \n', Ck, np.array(tk))
        trajk = t1.copy()
        trajk.set_coeffs(coeffs=Ck)
        trajk.plot(times=tk, ax=ax)
        
        tk = [t_n]
        Ck = None
        
print('print the latest one too.')
print('plotting \n', Ck, np.array(tk))
trajk = t1.copy()
trajk.set_coeffs(coeffs=Ck)
trajk.plot(times=tk, ax=ax)